In [1]:
# Loading Data
from src.data_preparation import load_data
import matplotlib.pyplot as plt
plt.ioff()
PROCESSED_DATA_PATH = "data/processed/df_1h_lag_BLV_spatial_images.csv"
df = load_data(PROCESSED_DATA_PATH, date_col="measurement_time")

df.drop(columns=["Unnamed: 0", "timestamp"], inplace=True)



df_interpolated = df.interpolate(method='linear')

df_interpolated.fillna(0, inplace=True)

features_to_check = ['ghi', 'dni', 'solar_zenith', 'GHI_cs', 'DNI_cs', 'CSI_ghi', 
                                   'CSI_dni',
                                   'nam_ghi', 'nam_dni', 'nam_cc', 'B_CSI_ghi_8h', 'V_CSI_ghi_8h', 'L_CSI_ghi_8h', 'B_CSI_ghi_9h',
       'V_CSI_ghi_9h', 'L_CSI_ghi_9h', 'B_CSI_ghi_10h', 'V_CSI_ghi_10h',
       'L_CSI_ghi_10h', 'B_CSI_ghi_11h', 'V_CSI_ghi_11h', 'L_CSI_ghi_11h',
       'B_CSI_ghi_12h', 'V_CSI_ghi_12h', 'L_CSI_ghi_12h', 'B_CSI_ghi_13h',
       'V_CSI_ghi_13h', 'L_CSI_ghi_13h', 'B_CSI_ghi_14h', 'V_CSI_ghi_14h',
       'L_CSI_ghi_14h', 'B_CSI_ghi_15h', 'V_CSI_ghi_15h', 'L_CSI_ghi_15h',
       'B_CSI_ghi_16h', 'V_CSI_ghi_16h', 'L_CSI_ghi_16h', 'B_CSI_ghi_17h',
       'V_CSI_ghi_17h', 'L_CSI_ghi_17h', 'B_CSI_ghi_18h', 'V_CSI_ghi_18h',
       'L_CSI_ghi_18h', 'B_CSI_ghi_19h', 'V_CSI_ghi_19h', 'L_CSI_ghi_19h',
       '80_dwsw', '80_cloud_cover', '56_dwsw', '56_cloud_cover',
       '20_dwsw', '20_cloud_cover', '88_dwsw', '88_cloud_cover', 'AVG(R)',
       'STD(R)', 'ENT(R)', 'AVG(G)', 'STD(G)', 'ENT(G)', 'AVG(B)', 'STD(B)',
       'ENT(B)', 'AVG(RB)', 'STD(RB)', 'ENT(RB)', 'AVG(NRB)', 'STD(NRB)',
       'ENT(NRB)']

# 2. Calculate the number of NaN values for each feature
# We use .isna() instead of == 0
nans_per_feature = df_interpolated[features_to_check].isna().sum()

# 3. Filter to get only features that actually have NaN values
features_with_nans = nans_per_feature[nans_per_feature > 0]

# 4. Report the findings for which features have NaNs
if features_with_nans.empty:
    print("No NaN (missing) values found in any of the specified features.")
else:
    print("--- Features With NaN Values ---")
    print("The following features have NaN values, with the total count for each:")
    # Sort for clearer output
    print(features_with_nans.sort_values(ascending=False))
    print("\n" + "="*40 + "\n")

    # 5. Analyze the distribution of hours for rows containing NaNs
    print("--- Distribution of NaN-Value Records by Hour ---")
    
    # Create a boolean mask for rows that contain *at least one* NaN
    # in the specified columns
    rows_with_any_nan = df_interpolated[features_to_check].isna().any(axis=1)
    
    if rows_with_any_nan.sum() > 0:
        # Get the index for these rows
        nan_rows_index = df_interpolated.index[rows_with_any_nan]
        
        # Extract the hour from the DatetimeIndex and get the value counts
        hour_distribution = nan_rows_index.hour.value_counts().sort_index()
        
        print("Distribution of records (rows) containing at least one NaN, by hour:")
        print(hour_distribution)
        
        # Optional: Print total number of affected rows
        print(f"\nTotal number of rows with at least one NaN: {rows_with_any_nan.sum()}")
    else:
        # This case shouldn't be hit if features_with_nans was not empty,
        # but it's good practice to include.
        print("No rows found with NaN values (this is unexpected, check logic).")


import torch
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if hasattr(torch.backends, "mps") and torch.backends.mps.is_available()
    else "cpu"
)

device


Loaded 11,560 records
Date range: 2014-01-03 14:00:00+00:00 to 2016-12-30 23:00:00+00:00
Timezone: UTC
No NaN (missing) values found in any of the specified features.


/tmp/ipykernel_1625306/1365827564.py:12: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_interpolated = df.interpolate(method='linear')


'cuda'

In [2]:
df.columns

Index(['ghi', 'dni', 'PST_Time_meas', 'solar_zenith', 'time_gap_hours',
       'time_gap_norm', 'day_boundary_flag', 'hour_progression',
       'absolute_hour', 'GHI_cs', 'DNI_cs', 'CSI_ghi', 'CSI_dni',
       'season_flag', 'hour_sin', 'hour_cos', 'month_sin', 'month_cos',
       'nam_ghi', 'nam_dni', 'nam_cc', 'PST_Time_nam', 'nam_target_time',
       'B_CSI_ghi_8h', 'V_CSI_ghi_8h', 'L_CSI_ghi_8h', 'B_CSI_ghi_9h',
       'V_CSI_ghi_9h', 'L_CSI_ghi_9h', 'B_CSI_ghi_10h', 'V_CSI_ghi_10h',
       'L_CSI_ghi_10h', 'B_CSI_ghi_11h', 'V_CSI_ghi_11h', 'L_CSI_ghi_11h',
       'B_CSI_ghi_12h', 'V_CSI_ghi_12h', 'L_CSI_ghi_12h', 'B_CSI_ghi_13h',
       'V_CSI_ghi_13h', 'L_CSI_ghi_13h', 'B_CSI_ghi_14h', 'V_CSI_ghi_14h',
       'L_CSI_ghi_14h', 'B_CSI_ghi_15h', 'V_CSI_ghi_15h', 'L_CSI_ghi_15h',
       'B_CSI_ghi_16h', 'V_CSI_ghi_16h', 'L_CSI_ghi_16h', 'B_CSI_ghi_17h',
       'V_CSI_ghi_17h', 'L_CSI_ghi_17h', 'B_CSI_ghi_18h', 'V_CSI_ghi_18h',
       'L_CSI_ghi_18h', 'B_CSI_ghi_19h', 'V_CSI_ghi_19h',

In [3]:
df.reset_index(inplace=True)

In [4]:
df

,measurement_time,ghi,dni,PST_Time_meas,solar_zenith,time_gap_hours,time_gap_norm,day_boundary_flag,hour_progression,absolute_hour,...,ENT(G),AVG(B),STD(B),ENT(B),AVG(RB),STD(RB),ENT(RB),AVG(NRB),STD(NRB),ENT(NRB)
0,2014-01-03 14:00:00+00:00,0.000000,0.000000,2014-01-03 06:29:30-08:00,100.276046,0.166667,0.006944,0.016667,6.491667,24.491667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-01-03 15:00:00+00:00,12.914500,89.245833,2014-01-03 07:29:30-08:00,89.724162,0.016667,0.000694,0.000000,7.491667,25.491667,...,5.564803,151.246458,28.520968,5.360367,0.808373,0.144332,4.688985,-0.113270,0.086132,4.008980
2,2014-01-03 16:00:00+00:00,143.775500,636.475000,2014-01-03 08:29:30-08:00,80.146724,0.016667,0.000694,0.000000,8.491667,26.491667,...,5.510188,141.713212,33.481258,5.414777,0.909988,0.148683,4.742913,-0.053787,0.079303,3.913008
3,2014-01-03 17:00:00+00:00,289.955000,783.193333,2014-01-03 09:29:30-08:00,71.988814,0.016667,0.000694,0.000000,9.491667,27.491667,...,5.478535,137.213417,35.403827,5.377718,0.965367,0.142668,4.686905,-0.022647,0.071275,3.777387
4,2014-01-03 18:00:00+00:00,402.643333,843.050000,2014-01-03 10:29:30-08:00,65.816158,0.016667,0.000694,0.000000,10.491667,28.491667,...,5.456043,138.616210,36.162608,5.356960,0.947050,0.140950,4.682247,-0.032250,0.072482,3.788302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11555,2016-12-30 19:00:00+00:00,433.891667,753.421667,2016-12-30 11:29:30-08:00,62.502377,0.016667,0.000694,0.000000,11.491667,26237.491667,...,5.545958,135.665343,42.458667,5.359012,0.849203,0.178163,4.844802,-0.092885,0.101738,4.111970
11556,2016-12-30 20:00:00+00:00,449.525000,747.595000,2016-12-30 12:29:30-08:00,62.079248,0.016667,0.000694,0.000000,12.491667,26238.491667,...,5.617600,128.237255,44.904030,5.412868,0.902123,0.172672,4.814140,-0.059772,0.093093,4.012365
11557,2016-12-30 21:00:00+00:00,410.876667,735.498333,2016-12-30 13:29:30-08:00,64.765899,0.016667,0.000694,0.000000,13.491667,26239.491667,...,5.643208,119.943472,42.973342,5.470670,0.880178,0.188148,4.893065,-0.074325,0.104203,4.123958
11558,2016-12-30 22:00:00+00:00,313.066667,684.428333,2016-12-30 14:29:30-08:00,70.203098,0.016667,0.000694,0.000000,14.491667,26240.491667,...,5.685198,123.339068,38.577063,5.459773,0.789770,0.209435,4.991567,-0.132832,0.127398,4.354853


In [5]:
import numpy as np
import pandas as pd
import pvlib

# 1) Ensure a clean UTC datetime index (these are your evaluation timestamps)
dfu = df.copy()
if "nam_target_time" in dfu.columns:
    dfu.index = pd.to_datetime(dfu["nam_target_time"], utc=True, errors="coerce")
elif "measurement_time" in dfu.columns:
    dfu.index = pd.to_datetime(dfu["measurement_time"], utc=True, errors="coerce")
else:
    if not isinstance(dfu.index, pd.DatetimeIndex):
        raise ValueError("Need a datetime column or DatetimeIndex.")
    dfu.index = dfu.index.tz_localize("UTC") if dfu.index.tz is None else dfu.index.tz_convert("UTC")
dfu = dfu.sort_index()
idx_eval = dfu.index  # <-- do NOT shift this

# 2) One clearsky for BOTH Actual & NAM (same site, same timestamps)
site = pvlib.location.Location(38.599891, -121.126680, tz="UTC", altitude=0.0)
ghi_cs = site.get_clearsky(idx_eval, model="ineichen")["ghi"].astype(float)

# 3) Actual & NAM GHI
true_ghi = (dfu["ghi"].astype(float)
            if "ghi" in dfu.columns
            else (dfu["CSI_ghi"].astype(float) * ghi_cs))

nam_ghi = dfu["nam_ghi"].astype(float)

# 4) Consistent CSI (same denominator)
with np.errstate(divide="ignore", invalid="ignore"):
    true_csi = (true_ghi / ghi_cs).replace([np.inf, -np.inf], np.nan).clip(0, 2)
    nam_csi  = (nam_ghi  / ghi_cs).replace([np.inf, -np.inf], np.nan).clip(0, 2)

# 5) Smart Persistence: value at time t equals yesterday’s actual at the SAME time t
#    shift the INDEX forward by 1 day (freq-shift), do NOT reindex against shifted labels
sp_ghi = true_ghi.shift(1, freq="1D")
sp_csi = true_csi.shift(1, freq="1D")

# 6) Align all series on the original evaluation timestamps (no filtering)
def align(s): return s.reindex(idx_eval).astype(float).values
y_true_csi = align(true_csi)
y_true_ghi = align(true_ghi)
y_nam_csi  = align(nam_csi)
y_nam_ghi  = align(nam_ghi)
y_sp_csi   = align(sp_csi)
y_sp_ghi   = align(sp_ghi)

# 7) Quick sanity checks (must NOT be identical)
if np.allclose(y_sp_ghi, y_true_ghi, equal_nan=False):
    raise RuntimeError("SP equals Actual; check the shift (use shift(freq='1D'), not reindex).")
if np.allclose(y_sp_csi, y_true_csi, equal_nan=False):
    raise RuntimeError("SP equals Actual; check the shift (use shift(freq='1D'), not reindex).")

# 8) Metrics & skill
def mae(a,b):
    a,b = np.asarray(a), np.asarray(b); m = np.isfinite(a)&np.isfinite(b)
    return float(np.mean(np.abs(a[m]-b[m]))) if m.any() else np.nan
def rmse(a,b):
    a,b = np.asarray(a), np.asarray(b); m = np.isfinite(a)&np.isfinite(b)
    return float(np.sqrt(np.mean((a[m]-b[m])**2))) if m.any() else np.nan
def skill(err_model, err_ref):
    return float(1 - err_model/err_ref) if np.isfinite(err_model) and np.isfinite(err_ref) and err_ref!=0 else np.nan

metrics = {
    "CSI": {
        "MAE_NAM":  mae(y_true_csi, y_nam_csi),
        "RMSE_NAM": rmse(y_true_csi, y_nam_csi),
        "MAE_SP":   mae(y_true_csi, y_sp_csi),
        "RMSE_SP":  rmse(y_true_csi, y_sp_csi),
    },
    "GHI": {
        "MAE_NAM":  mae(y_true_ghi, y_nam_ghi),
        "RMSE_NAM": rmse(y_true_ghi, y_nam_ghi),
        "MAE_SP":   mae(y_true_ghi, y_sp_ghi),
        "RMSE_SP":  rmse(y_true_ghi, y_sp_ghi),
    }
}
for k in ("CSI","GHI"):
    metrics[k]["Skill_NAM_vs_SP_MAE"]  = skill(metrics[k]["MAE_NAM"],  metrics[k]["MAE_SP"])
    metrics[k]["Skill_NAM_vs_SP_RMSE"] = skill(metrics[k]["RMSE_NAM"], metrics[k]["RMSE_SP"])

print(metrics)

{'CSI': {'MAE_NAM': 0.2713719362249024, 'RMSE_NAM': 0.4251471062903225, 'MAE_SP': 0.17177602424216554, 'RMSE_SP': 0.30032714669080457, 'Skill_NAM_vs_SP_MAE': -0.5798010078655043, 'Skill_NAM_vs_SP_RMSE': -0.4156133102680313}, 'GHI': {'MAE_NAM': 89.55589988602819, 'RMSE_NAM': 124.70897044773457, 'MAE_SP': 72.83015079920663, 'RMSE_SP': 136.77357957976824, 'Skill_NAM_vs_SP_MAE': -0.22965418721889774, 'Skill_NAM_vs_SP_RMSE': 0.08820862310617106}}


In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# ==== CONFIG ====
OUTDIR = Path("plots_nam_sp_vs_actual")
OUTDIR.mkdir(parents=True, exist_ok=True)
plt.ioff()  # prevent inline figures from printing in Jupyter

# ---- 1) Build aligned comparison frame ----
cmp = pd.DataFrame({
    "true_GHI": y_true_ghi,
    "NAM_GHI":  y_nam_ghi,
    "SP_GHI":   y_sp_ghi,
    "true_CSI": y_true_csi,
    "NAM_CSI":  y_nam_csi,
    "SP_CSI":   y_sp_csi,
}, index=pd.DatetimeIndex(idx_eval, name="time"))

# Optional: ensure numeric and drop rows that are fully NaN across the three streams
cmp = cmp.apply(pd.to_numeric, errors="coerce")

# ---- 2) Helper: pick a 3-day window with data ----
def pick_3day_window(df, cols, min_points=8):
    # Find the first date where we have enough finite points, then take that date + next 2 days
    day = None
    for d, g in df.groupby(df.index.normalize()):
        if np.isfinite(g[cols].values).sum() >= min_points:
            day = d
            break
    if day is None:
        # fallback: just take first 3 days from start
        day = df.index[0].normalize()
    start = day
    end   = (day + pd.Timedelta(days=3)) - pd.Timedelta(seconds=1)
    return slice(start, end)

win_ghi = pick_3day_window(cmp, ["true_GHI","NAM_GHI","SP_GHI"])
win_csi = pick_3day_window(cmp, ["true_CSI","NAM_CSI","SP_CSI"])

# ---- 3) Plot helpers (no return to avoid printing) ----
def plot_overlay(df, cols, title, outfile):
    plt.figure(figsize=(14,4))
    for c in cols:
        plt.plot(df.index, df[c], label=c)
    plt.title(title)
    plt.xlabel("Time"); plt.ylabel(cols[0].split("_")[1])  # crude 'GHI'/'CSI'
    plt.legend(); plt.tight_layout()
    plt.savefig(outfile, dpi=160)
    plt.close()

def plot_scatter(y_true, y_pred, title, outfile):
    plt.figure(figsize=(4.8,4.8))
    m = np.isfinite(y_true) & np.isfinite(y_pred)
    yt, yp = y_true[m], y_pred[m]
    plt.scatter(yt, yp, s=6, alpha=0.5)
    if len(yt) and len(yp):
        vmin = float(np.nanmin(np.concatenate([yt, yp])))
        vmax = float(np.nanmax(np.concatenate([yt, yp])))
        if np.isfinite(vmin) and np.isfinite(vmax):
            plt.plot([vmin, vmax], [vmin, vmax])
            plt.xlim(vmin, vmax); plt.ylim(vmin, vmax)
    plt.title(title); plt.xlabel("Actual"); plt.ylabel("Predicted")
    plt.tight_layout(); plt.savefig(outfile, dpi=160); plt.close()

def plot_bars(mae_nam, mae_sp, rmse_nam, rmse_sp, title, outfile):
    dfb = pd.DataFrame(
        {"NAM":[mae_nam, rmse_nam], "SP":[mae_sp, rmse_sp]},
        index=["MAE","RMSE"]
    )
    ax = dfb.plot(kind="bar", figsize=(6,4))
    ax.set_title(title); ax.set_xlabel("")
    for p in ax.patches:
        ax.annotate(f"{p.get_height():.1f}", (p.get_x()+p.get_width()/2, p.get_height()),
                    ha="center", va="bottom", xytext=(0,2), textcoords="offset points", fontsize=8)
    plt.tight_layout(); plt.savefig(outfile, dpi=160); plt.close()

# ---- 4) Three 3-day overlays for GHI ----
sub_ghi = cmp.loc[win_ghi, ["true_GHI","NAM_GHI","SP_GHI"]]
plot_overlay(sub_ghi, ["true_GHI","NAM_GHI","SP_GHI"],
             "GHI – Actual vs NAM vs SP (3 days)",
             OUTDIR/"overlay_ghi_all_3days.png")
plot_overlay(sub_ghi, ["true_GHI","NAM_GHI"],
             "GHI – Actual vs NAM (3 days)",
             OUTDIR/"overlay_ghi_nam_3days.png")
plot_overlay(sub_ghi, ["true_GHI","SP_GHI"],
             "GHI – Actual vs Smart Persistence (3 days)",
             OUTDIR/"overlay_ghi_sp_3days.png")

# ---- 5) Three 3-day overlays for CSI ----
sub_csi = cmp.loc[win_csi, ["true_CSI","NAM_CSI","SP_CSI"]]
plot_overlay(sub_csi, ["true_CSI","NAM_CSI","SP_CSI"],
             "CSI – Actual vs NAM vs SP (3 days)",
             OUTDIR/"overlay_csi_all_3days.png")
plot_overlay(sub_csi, ["true_CSI","NAM_CSI"],
             "CSI – Actual vs NAM (3 days)",
             OUTDIR/"overlay_csi_nam_3days.png")
plot_overlay(sub_csi, ["true_CSI","SP_CSI"],
             "CSI – Actual vs Smart Persistence (3 days)",
             OUTDIR/"overlay_csi_sp_3days.png")

# ---- 6) Whole-dataset scatter plots ----
plot_scatter(cmp["true_GHI"].values, cmp["NAM_GHI"].values,
             "GHI – NAM vs Actual", OUTDIR/"scatter_ghi_nam.png")
plot_scatter(cmp["true_GHI"].values, cmp["SP_GHI"].values,
             "GHI – SP vs Actual", OUTDIR/"scatter_ghi_sp.png")
plot_scatter(cmp["true_CSI"].values, cmp["NAM_CSI"].values,
             "CSI – NAM vs Actual", OUTDIR/"scatter_csi_nam.png")
plot_scatter(cmp["true_CSI"].values, cmp["SP_CSI"].values,
             "CSI – SP vs Actual", OUTDIR/"scatter_csi_sp.png")

# ---- 7) Error bars (use your computed metrics dict) ----
plot_bars(metrics["GHI"]["MAE_NAM"], metrics["GHI"]["MAE_SP"],
          metrics["GHI"]["RMSE_NAM"], metrics["GHI"]["RMSE_SP"],
          "GHI Errors (lower is better)", OUTDIR/"bar_ghi_errors.png")

plot_bars(metrics["CSI"]["MAE_NAM"], metrics["CSI"]["MAE_SP"],
          metrics["CSI"]["RMSE_NAM"], metrics["CSI"]["RMSE_SP"],
          "CSI Errors (lower is better)", OUTDIR/"bar_csi_errors.png")

print("Saved plots to:", OUTDIR.resolve())

Saved plots to: /home/muhammadhassan/App_v02/plots_nam_sp_vs_actual


---